In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

In [119]:
ind = pd.read_csv("indicators2.csv")
ind_drop = ind.drop(["Country Name","Series Code"], axis = 1)
ind_melt = ind_drop.melt(id_vars=['Region','Series Name'], var_name= "Year",value_name= "Value")
ind_melt['Year'] = ind_melt['Year'].str.split(' ',expand= True)[0]
indicators = ind_melt.replace("..", np.NaN).dropna()
indicators['Value'] = indicators['Value'].astype(float).round(3)
group = indicators.groupby(['Region','Year','Series Name']).mean().reset_index()
Ind = group.pivot(index = ['Region','Year'],columns= 'Series Name', values= 'Value').reset_index()

In [120]:
cci = pd.read_csv("https://richardson.byu.edu/data_contest/CCI.csv")
cci['Year'] = cci['TIME'].str.split("-",expand= True)[0]
cci_drop = cci.drop(['TIME'], axis = 1)
CCI = cci_drop.groupby(['LOCATION','Year']).mean().reset_index()
CCI.rename(columns={'LOCATION': 'Region'},inplace= True)

In [122]:
econ = Ind.merge(CCI, on = ['Year','Region'], how= 'outer')
econ

,Region,Year,Consumer price index (2010 = 100),GDP per capita (current US$),GDP per capita growth (annual %),"Inflation, consumer prices (annual %)",Lending interest rate (%),"Unemployment, total (% of total labor force) (national estimate)",CCI
0,GLOBAL,1960,7.222077,476.232835,NaN,3.554986,5.44475,3.60,NaN
1,GLOBAL,1961,7.250746,489.201548,1.149325,3.491319,5.60925,4.05,NaN
2,GLOBAL,1962,7.447397,513.139270,2.693039,4.630811,5.47900,3.40,NaN
3,GLOBAL,1963,7.737097,544.597905,2.516859,5.768720,5.21875,3.50,NaN
4,GLOBAL,1964,7.716247,590.847891,3.896414,6.263899,5.62300,3.15,NaN
...,...,...,...,...,...,...,...,...,...
232,USA,2018,NaN,NaN,NaN,NaN,NaN,NaN,101.459800
233,USA,2019,NaN,NaN,NaN,NaN,NaN,NaN,101.231775
234,USA,2020,NaN,NaN,NaN,NaN,NaN,NaN,99.387968
235,USA,2021,NaN,NaN,NaN,NaN,NaN,NaN,98.939459


In [79]:
games = pd.read_csv("vgsales.csv")
games['Year'] = pd.to_datetime(games['Year'], format = '%Y').dt.strftime('%Y')
games

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [99]:
econ.merge(games, on = 'Year')

,Country Code,Year,Consumer price index (2010 = 100),GDP per capita (current US$),GDP per capita growth (annual %),"Inflation, consumer prices (annual %)",Lending interest rate (%),"Unemployment, total (% of total labor force) (national estimate)",CCI,Rank,Name,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,JPN,1982,83.172,9575.608,2.566,2.741,NaN,2.4,100.156079,90,Pac-Man,2600,Puzzle,Atari,7.28,0.45,0.0,0.08,7.81
1,JPN,1982,83.172,9575.608,2.566,2.741,NaN,2.4,100.156079,1117,Dig Dug,2600,Puzzle,Atari,1.52,0.10,0.0,0.02,1.64
2,JPN,1982,83.172,9575.608,2.566,2.741,NaN,2.4,100.156079,1165,Mario Bros.,2600,Platform,Atari,1.48,0.09,0.0,0.02,1.59
3,JPN,1982,83.172,9575.608,2.566,2.741,NaN,2.4,100.156079,1707,Pole Position,2600,Racing,Atari,1.10,0.07,0.0,0.01,1.18
4,JPN,1982,83.172,9575.608,2.566,2.741,NaN,2.4,100.156079,1819,Moon Patrol,2600,Shooter,Atari,1.05,0.06,0.0,0.01,1.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48921,USA,1981,41.698,13976.105,1.536,10.335,18.87,7.6,98.096134,7186,Crazy Climber,2600,Action,Atari,0.21,0.01,0.0,0.00,0.22
48922,USA,1981,41.698,13976.105,1.536,10.335,18.87,7.6,98.096134,7373,RealSports Baseball,2600,Sports,Atari,0.20,0.01,0.0,0.00,0.21
48923,USA,1981,41.698,13976.105,1.536,10.335,18.87,7.6,98.096134,7906,International Soccer,2600,Sports,Mattel Interactive,0.18,0.01,0.0,0.00,0.19
48924,USA,1981,41.698,13976.105,1.536,10.335,18.87,7.6,98.096134,8461,Armor Ambush,2600,Action,Mattel Interactive,0.15,0.01,0.0,0.00,0.17
